## Import libraries

> Install them with `pip install -r requirements.txt`

In [ ]:
import urbanpy as up
import contextily as cx
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm.auto import tqdm

In [ ]:
# Activate tqdm progress bar for pandas.apply
tqdm.pandas()

## Get city administrative limits

In [ ]:
pos_id = 0  # Result position
riyad = up.download.nominatim_osm("Riyadh Governorate, Saudi Arabia", pos_id)  # Nominatim query

In [ ]:
# Plot administrative limits
ax = riyad.plot(facecolor="none", edgecolor="r")
# Add a basemap
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")
ax.set_axis_off()

# Get city population density 

In [ ]:
# Query the population data in Saudi Arabia from the repository of Meta Data For Good Population Maps in the Humanitarian Data Exchange platform
population_search_results = up.download.search_hdx_dataset("Saudi Arabia")

In [ ]:
population_search_results

In [ ]:
# We selected the index 9 that refers to the Elderly (ages 60+)
saudi_arabia_pop = up.download.get_hdx_dataset(population_search_results, 9)
saudi_arabia_pop.head()

In [ ]:
# Number of population points for the country
saudi_arabia_pop.shape[0]

In [ ]:
# Use the city adm limits to filter the country population data
filtered_pop = up.geom.filter_population(saudi_arabia_pop, riyad)
filtered_pop.head()

In [ ]:
# Number of population points for the city
filtered_pop.shape[0]

In [ ]:
# Plot population points
ax = filtered_pop.plot("sau_elderly_60_plus_2020", markersize=0.01, legend=True)
# Plot administrative limit
riyad.plot(facecolor="none", edgecolor="r", ax=ax)
# Add a basemap
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")
ax.set_axis_off()

## To improve the interpretability of the population plot we will group them in uniform spatial units known as [H3 Hexagons](https://h3geo.org/docs/).

In [ ]:
# Generate hexagons of resolution 7 (~5.1612km2)
riyad_hexs = up.geom.gen_hexagons(resolution=7, city=riyad)

> [See the table of resolution and sizes here](https://h3geo.org/docs/core-library/restable#average-area-in-km2)

In [ ]:
# Plot the city H3 hexagons
ax = riyad_hexs.plot(facecolor="none", edgecolor="r")
# Add a basemap
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")
ax.set_axis_off()

In [ ]:
# Sum the point's population within each hexagons
merges_hexs = up.geom.merge_shape_hex(
    riyad_hexs, filtered_pop, {"sau_elderly_60_plus_2020": "sum"}
)  # You can use other aggregation methods like max, min, count, and mean

In [ ]:
# Plot hexagons colored by population density
ax = merges_hexs.plot("sau_elderly_60_plus_2020", legend=True, missing_kwds={"color": "grey"})
# Add a basemap
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")
ax.set_axis_off()

In [ ]:
# Get amenities (Point of Interest) from Overpass
gdf_pois, _ = up.download.overpass(type_of_data="node", query={"amenity": ["clinic", "hospital"]}, mask=riyad)
gdf_pois.head()

> [See all types of data you can query from Overpass](https://wiki.openstreetmap.org/wiki/Map_features)

In [ ]:
# Plot points of interests
ax = gdf_pois.plot("poi_type", legend=True, figsize=(10, 10))
# Add a basemap
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")
ax.set_axis_off()

In [ ]:
# Start routing server (needs docker)
up.routing.start_osrm_server("gcc-states", "asia", "car")  # foot,car,bicycle

In [ ]:
# Calculate travel times (duration in minutes and distance in km) from hexagons to points of interest
merges_hexs_tt = up.accessibility.travel_times(merges_hexs, gdf_pois)

In [ ]:
up.routing.stop_osrm_server("gcc-states", "asia", "car")

In [ ]:
merges_hexs_tt.head()

In [ ]:
# Plot distance and duration maps
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

# Plot distance
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=-0.1)
merges_hexs_tt.plot("distance_to_nearest_poi", legend=True, ax=ax1, cax=cax)
cx.add_basemap(ax1, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")
ax1.set_axis_off()
ax1.set_title("Distance to Nearest PoI")
# Plot duration
merges_hexs_tt.plot("duration_to_nearest_poi_label", cmap="magma_r", legend=True, ax=ax2)
# Add a basemap
cx.add_basemap(ax2, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")
ax2.set_axis_off()
ax2.set_title("Duration to Nearest PoI")

plt.show()

## Generate interactive maps

In [ ]:
plotly.offline.init_notebook_mode()

In [ ]:
fig = up.plotting.choropleth_map(
    merges_hexs_tt, "sau_elderly_60_plus_2020", title="Estimated Elderly (60+) Population - 2020", opacity=0.5
)

# Remove the hexagon outlines to make the map clearer
fig.update_traces(marker_line_width=0)

# Make space for the title
fig.update_layout(margin=dict(l=0, r=0, b=0))

In [ ]:
# Filter out the hexagons without population
merges_hexs_tt_filtered_pop = merges_hexs_tt.query("sau_elderly_60_plus_2020 > 0").reset_index(drop=True)

In [ ]:
# Get ordered category labels
category_orders = merges_hexs_tt_filtered_pop["duration_to_nearest_poi_label"].unique().sort_values()

In [ ]:
fig = up.plotting.choropleth_map(
    merges_hexs_tt_filtered_pop,
    color_column="duration_to_nearest_poi_label",
    color_discrete_sequence=px.colors.sequential.Plasma_r,
    category_orders={"duration_to_nearest_poi_label": category_orders},
    labels={"duration_to_nearest_poi_label": "Minutes"},
    title="Travel Time to Nearest PoI",
    opacity=0.5,
)

# Make space for the title
fig.update_layout(margin=dict(l=0, r=0, b=0))

# Remove the hexagon outlines to make the map clearer
fig.update_traces(marker_line_width=0)